**AI-Driven Generation of Compliance Reports** 

This script generates an automated HTML compliance report using the Jinja2 templating engine. It first sets up the environment and checks if a predefined HTML template exists; if not, it uses an inline default template. The generate_report function takes environmental data (like emissions, anomalies, and compliance scores), prepares statistics, creates plots using matplotlib , and fills the template with the provided and processed data. It then saves the final rendered report as an HTML file. This setup enables easy generation of professional, data-driven reports with visualizations and summaries, useful for monitoring compliance or auditing environmental performance.

In [ ]:
from jinja2 import Environment, FileSystemLoader
import base64
from io import BytesIO
import matplotlib.pyplot as plt
from datetime import datetime
import os


env = Environment(loader=FileSystemLoader('.'), autoescape=True)

template_file = 'report_template.html'
template_exists = os.path.exists(template_file)

# Comprehensive template with multiple sections
if template_exists:
    template = env.get_template(template_file)
else:
    template = env.from_string("""
<!DOCTYPE html>
<html>
<head>
    <title>Compliance Report: {{ report_date }}</title>
    <style>
        body { font-family: Arial, sans-serif; line-height: 1.6; margin: 0 auto; max-width: 1200px; padding: 20px; }
        .header { background-color: #f5f5f5; padding: 20px; border-radius: 5px; margin-bottom: 20px; }
        .section { margin-bottom: 30px; border-bottom: 1px solid #eee; padding-bottom: 20px; }
        .section-title { color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px; }
        .stat-grid { display: grid; grid-template-columns: repeat(auto-fill, minmax(200px, 1fr)); gap: 15px; }
        .stat-card { background: #f9f9f9; padding: 15px; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
        .stat-value { font-size: 24px; font-weight: bold; color: #2c3e50; }
        .anomaly { background: #fff8e1; padding: 10px; margin: 10px 0; border-left: 4px solid #ffc107; }
        .plot-container { text-align: center; margin: 20px 0; }
        .plot { max-width: 100%; height: auto; }
        .footer { margin-top: 40px; font-size: 0.9em; color: #7f8c8d; text-align: center; }
    </style>
</head>
<body>
    <div class="header">
        <h1>AI Compliance Report</h1>
        <p>Generated on: {{ generation_time }}</p>
        <p>Reporting period: {{ report_date }}</p>
    </div>

    <div class="section">
        <h2 class="section-title">Executive Summary</h2>
        <p>{{ summary_text }}</p>
        
        <div class="stat-grid">
            {% for stat in summary_stats %}
            <div class="stat-card">
                <h3>{{ stat.name }}</h3>
                <p class="stat-value">{{ stat.value }}</p>
                <p>{{ stat.description }}</p>
            </div>
            {% endfor %}
        </div>
    </div>

    <div class="section">
        <h2 class="section-title">Key Metrics Visualization</h2>
        <div class="plot-container">
            <img src="data:image/png;base64,{{ main_plot }}" class="plot" alt="Main Metrics Visualization">
        </div>
    </div>

    <div class="section">
        <h2 class="section-title">Anomaly Detection</h2>
        <p>AI-detected {{ anomalies|length }} potential compliance issues:</p>
        
        {% for anomaly in anomalies %}
        <div class="anomaly">
            <h3>Issue #{{ loop.index }}: {{ anomaly.type }}</h3>
            <p><strong>Location:</strong> {{ anomaly.location }}</p>
            <p><strong>Detected:</strong> {{ anomaly.timestamp }}</p>
            <p><strong>Severity:</strong> {{ anomaly.severity }}</p>
            <p><strong>Description:</strong> {{ anomaly.description }}</p>
            {% if anomaly.recommendation %}
            <p><strong>Recommendation:</strong> {{ anomaly.recommendation }}</p>
            {% endif %}
        </div>
        {% endfor %}
        
        {% if trend_plot %}
        <div class="plot-container">
            <img src="data:image/png;base64,{{ trend_plot }}" class="plot" alt="Anomaly Trend Visualization">
        </div>
        {% endif %}
    </div>

    <div class="section">
        <h2 class="section-title">Regulatory References</h2>
        <ul>
            {% for regulation in regulations %}
            <li>
                <strong>{{ regulation.code }}:</strong> {{ regulation.description }}
                {% if regulation.section %} (Section {{ regulation.section }}){% endif %}
            </li>
            {% endfor %}
        </ul>
    </div>

    <div class="footer">
        <p>This report was automatically generated by AI Compliance Auditor v1.0</p>
        <p>Confidential - For internal use only</p>
    </div>
</body>
</html>
""")

def generate_plot(data, title, plot_type='bar'):
    """Generate a matplotlib plot and return as base64 string"""
    plt.figure(figsize=(10, 6))
    
    if plot_type == 'bar':
        plt.bar(data.keys(), data.values())
    elif plot_type == 'line':
        plt.plot(list(data.keys()), list(data.values()), marker='o')
    elif plot_type == 'pie':
        plt.pie(data.values(), labels=data.keys(), autopct='%1.1f%%')
    
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=120)
    buf.seek(0)
    img_b64 = base64.b64encode(buf.read()).decode('utf-8')
    plt.close()
    
    return img_b64

def generate_report(data):
    """Generate a comprehensive compliance report"""
    # Process data and prepare variables for template
    report_date = data.get('report_date', datetime.now().strftime('%B %Y'))
    generation_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Prepare summary statistics
    summary_stats = [
        {'name': 'Total Emissions', 'value': f"{data['total_emissions']:,.2f} tons", 'description': 'CO2 equivalent'},
        {'name': 'Anomalies Detected', 'value': len(data['anomalies']), 'description': 'Potential compliance issues'},
        {'name': 'Facilities', 'value': data.get('facility_count', 'N/A'), 'description': 'Included in report'},
        {'name': 'Compliance Score', 'value': f"{data.get('compliance_score', 0)}%", 'description': 'Based on regulatory requirements'}
    ]
    
    # Generate plots
    main_plot_data = {
        'Emissions': data['total_emissions'],
        'Threshold': data.get('threshold', data['total_emissions'] * 0.8)
    }
    main_plot = generate_plot(main_plot_data, 'Emissions vs Threshold')
    
    # Generate trend plot if historical data exists
    trend_plot = None
    if 'historical_emissions' in data:
        trend_plot = generate_plot(
            data['historical_emissions'],
            'Emissions Trend (6 Months)',
            plot_type='line'
        )
    
    # Prepare regulatory references
    regulations = [
        {'code': 'EPA 40 CFR Part 98', 'description': 'Mandatory Greenhouse Gas Reporting', 'section': '98.3'},
        {'code': 'EU ETS', 'description': 'European Union Emissions Trading System', 'section': 'Annex I'},
        {'code': 'ISO 14064-1', 'description': 'Greenhouse gas accounting for organizations'}
    ]
    
    # Render template with all data
    html = template.render(
        report_date=report_date,
        generation_time=generation_time,
        summary_text=data.get('summary_text', 'Automated summary of compliance metrics and anomalies.'),
        summary_stats=summary_stats,
        main_plot=main_plot,
        trend_plot=trend_plot,
        anomalies=data['anomalies'],
        regulations=regulations
    )
    
    # Save report
    filename = f"compliance_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
    with open(filename, "w") as f:
        f.write(html)
    
    return filename

# Example usage 
if __name__ == "__main__":
    sample_data = {
        'report_date': 'Q1 2025',
        'total_emissions': 12345.67,
        'facility_count': 8,
        'compliance_score': 92.5,
        'threshold': 10000,
        'historical_emissions': {
            'Oct 2024': 9876,
            'Nov 2024': 10543,
            'Dec 2024': 11234,
            'Jan 2025': 12345,
            'Feb 2025': 11876,
            'Mar 2025': 12567
        },
        'anomalies': [
            {
                'type': 'Unusual Emission Spike',
                'location': 'Facility A-12',
                'timestamp': '2025-03-15 14:30',
                'severity': 'High',
                'description': 'Emission levels exceeded normal operating range by 42%',
                'recommendation': 'Inspect equipment and review operational logs'
            },
            {
                'type': 'Missing Data',
                'location': 'Facility B-7',
                'timestamp': '2025-03-18 09:15',
                'severity': 'Medium',
                'description': 'Missing 8 hours of emissions data due to sensor malfunction'
            }
        ],
        'summary_text': 'This quarter shows a 12% increase in emissions compared to Q4 2024. Two significant anomalies were detected requiring investigation.'
    }
    
    report_file = generate_report(sample_data)
    print(f"Report generated: {report_file}")

Report generated: compliance_report_20250608_173140.html
